In [1]:
import tweepy
import json
from pymongo import MongoClient
import datetime
from textblob import TextBlob

Enter required keys from https://apps.twitter.com/ in quotes

In [2]:
consumer_key=' '
consumer_secret =' '
access_token = ' '
access_token_secret = ' '

Enter the Connection name and collection name. If it doesn't exist, MongoDB will create it. Here connection is 'TwitterData' and collection is 'cricket'.

In [8]:
connection=MongoClient('localhost',27017)
db=connection.TwitterData
db.cricket.create_index("id",unique=True,dropDups=True)
collection=db.cricket

In [9]:
print ('Enter keywords separated by comma. Do not use space after comma')
keywords=map(str,raw_input().split(','))
language=['en']

Enter keywords separated by comma. Do not use space after comma
cricket,Cricket


In [10]:
print('Do you want to collect retweets? [Y/N]')
ans=raw_input()

Do you want to collect retweets? [Y/N]
N


You can define the labels according to their sentiment polarity in this function.

In [11]:
def sentiment(text2):
    global score,result
    blob=TextBlob(text2)
    score=blob.sentiment.polarity
    if 0.1 >= score > -0.1:
        result='neutral'
    elif 0.7 >= score > 0.1:
        result='positive'
    elif score > 0.7:
        result='highly positive'
    elif -0.1 >= score > -0.7:
        result='negative'
    elif score <= -0.7 :
        result = 'highly negative'
    return score,result

In [ ]:
class MyStreamListener(tweepy.StreamListener):
    def on_data(self, data):
        tweet=json.loads(data)    
        if ans=='N':    
            if (not tweet['retweeted']) and ('RT @' not in tweet['text']):
                if "extended_tweet" in tweet:
                    text=tweet['extended_tweet']['full_text'].encode('ascii', 'ignore')
                else:
                    text=tweet['text'].encode('ascii', 'ignore')    
                tweet_id=tweet['id_str']
                username=tweet['user']['screen_name']
                followers=tweet['user']['followers_count']
                #blob=TextBlob(text)
                sentiment_scores=sentiment(text)[0]
                tweet_sentiment=sentiment(text)[1]
                RT_count=tweet['retweet_count']
                quote_count=tweet['quote_count']
                fav_count=tweet['favorite_count']
                reply_count=tweet['reply_count']
                hashtags=tweet['entities']['hashtags']
                dt=tweet['created_at']
                created_at=datetime.datetime.strptime(dt,'%a %b %d %H:%M:%S +0000 %Y')
                final_tweet={'id':tweet_id,'handle':username,'followers':followers,'text':text,
                             'RTs':RT_count,'Quotes':quote_count,'Favourited':fav_count,'Replied':reply_count,
                             'hashtags':hashtags,
                             'Timestamp':created_at,'Sentiment_Score':sentiment_scores,'Tweet_Sentiment':tweet_sentiment}
                collection.insert_one(final_tweet)
                print (username + ':' + ' ' + text)
                #print final_tweet
                return True
                #print tweet
                #print ('%s: %s' % (tweet['user']['screen_name'], tweet['text'].encode('ascii', 'ignore')))
        elif ans=='Y':
            if "extended_tweet" in tweet:
                    text=tweet['extended_tweet']['full_text'].encode('ascii', 'ignore')
            else:
                text=tweet['text'].encode('ascii', 'ignore')    
               
            tweet_id=tweet['id_str']
            username=tweet['user']['screen_name']
            followers=tweet['user']['followers_count']
            #blob=TextBlob(text)
            sentiment_scores=sentiment(text)[0]
            tweet_sentiment=sentiment(text)[1]
            RT_count=tweet['retweet_count']
            quote_count=tweet['quote_count']
            fav_count=tweet['favorite_count']
            reply_count=tweet['reply_count']
            hashtags=tweet['entities']['hashtags']
            dt=tweet['created_at']
            created_at=datetime.datetime.strptime(dt,'%a %b %d %H:%M:%S +0000 %Y')
            final_tweet={'id':tweet_id,'handle':username,'followers':followers,'text':text,
                         'RTs':RT_count,'Quotes':quote_count,'Favourited':fav_count,'Replied':reply_count,
                         'hashtags':hashtags,
                         'Timestamp':created_at,'Sentiment_Score':sentiment_scores,'Tweet_Sentiment':tweet_sentiment}
            collection.insert_one(final_tweet)
            print (username + ':' + ' ' + text)
            #print final_tweet
            return True
            #print tweet
            #print ('%s: %s' % (tweet['user']['screen_name'], tweet['text'].encode('ascii', 'ignore')))
        else:
            print ('Please answer as Y for yes and N for no')
            
    def on_error(self,status):
        print(status)
if __name__ == '__main__':
    mylistener=MyStreamListener()
    print ('Printing Tweets :')
    consumer_auth=tweepy.OAuthHandler(consumer_key,consumer_secret)
    consumer_auth.set_access_token(access_token,access_token_secret)
    stream=tweepy.Stream(auth=consumer_auth,listener=mylistener,tweet_mode= 'extended')
    stream.filter(track=keywords,languages=language)

Printing Tweets :
mishnion: @kimrhodes4real You watching cricket for first time has got to be the best thing ever
JoanneFaulkne17: Just dont look over there at Manus and Nauru... and definitely dont look at our treatment of Aboriginal kids in detention on the mainland. https://t.co/DNIs3oYshI
johnnol1: Ball tampering: The staggering financial cost for a day of Cape Town madness | The New Daily https://t.co/76XoeKG9Ju
sunbeam007: @yelobeli @bbctms @Aggerscricket https://t.co/tOxwtcHYtI Got to have an option of someone who can bowl 145-150 and stay in the 140s. A southpaw is a bonus. Imagine the buzz if Garton and Mahmood both got a start.
mmbrenn: @MikeCarlton01 @allieannie1 No  He was, simply, weak  As captain of the Australian Cricket team to turn a blind eye to blatant cheating, showed a serious lapse in judgement and leadership.  But nevertheless, forgivable.
afeef1915: Ball-Tampering Scandal: Video Surfaces Of Pat Cummins Stepping On The Ball With Spikes. Not Tampering, Was It? htt